In [2]:
import pandas as pd
import folium

In [6]:
folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter")

folium.CircleMarker(location=[40.738, -73.98],fill=True).add_to(folium_map)
folium_map


bike_data  = pd.read_csv("/home/lucifer/Documents/Git/MMDP/data/13_01_01_added.csv")
bike_data["pickup_time"] = pd.to_datetime(bike_data["pickup_time"])
bike_data["drop_time"] = pd.to_datetime(bike_data["drop_time"])
# bike_data["hour"] = bike_data["Start Time"].map(lambda x: x.hour)

In [7]:
bike_data.head()

,taxi_id,driver_id,un1,un2,un5,pickup_time,drop_time,un3,travel_time,dist,pickup_lon,pickup_lat,drop_lon,drop_lat,base_fare,tax1,tax2,tax3,tax4,final_fare
0,76942C3205E17D7E7FE5A9F709D16434,25BA06A87905667AA1FE5990E33F0E2E,VTS,1,NaN,2013-01-01,2013-01-01 00:05:00,3,300,0.61,-73.955925,40.781887,-73.963181,40.777832,5.0,0.5,0.5,0.00,0.0,6.00
1,4A9DED62DD8EA1E1988C55F7141DC1F7,F4B5F923554A0BB822F26FCB25923D67,VTS,1,NaN,2013-01-01,2013-01-01 00:09:00,1,540,2.14,-73.975723,40.781693,-73.969147,40.760975,9.5,0.5,0.5,3.33,0.0,13.83
2,BB4FB2B3BB0C4FF05A28E8A1B269CB2C,21EF484FBA1254F1C1A721CD6EA979C1,VTS,1,NaN,2013-01-01,2013-01-01 00:16:00,4,960,4.05,-74.000526,40.737343,-73.977226,40.783607,14.5,0.5,0.5,4.65,0.0,20.15
3,8B5F45807D8EC24DDFD9A7EDFFAEC138,577D1B6729EF9014CDBC949554EB4483,VTS,1,NaN,2013-01-01,2013-01-01 00:05:00,1,300,0.06,-73.984871,40.753723,-73.983849,40.754467,4.5,0.5,0.5,0.00,0.0,5.50
4,E3ADD882D2B3BD1B1A583F1EF4A58BFC,1F14C43D3C8C222FD13F6537197055AD,VTS,1,NaN,2013-01-01,2013-01-01 00:08:00,2,480,1.68,-73.981575,40.767632,-73.977737,40.757927,8.0,0.5,0.5,1.70,0.0,10.70


In [12]:
def get_trip_counts_by_hour(selected_hour):
    # make a DataFrame with locations for each bike station
    locations = bike_data.groupby("drop_time").first()
    locations = locations.loc[:, ["pickup_lon",
                                 "pickup_lat"]]
    
    #select one time of day
    subset = bike_data[bike_data["un3"]==selected_hour]
    
    # count trips for each destination
    departure_counts =  subset.groupby("pickup_lon").count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["pickup_lat"]
    
    # count trips for each origin
    arrival_counts =  subset.groupby("drop_time").count().iloc[:,[0]]
    arrival_counts.columns= ["dist"]

    #join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    return trip_counts

# print a sample to check our code works
get_trip_counts_by_hour(6).head()

ValueError: columns overlap but no suffix specified: Index(['pickup_lat'], dtype='object')